In [4]:
import numpy as np
import k3d
from math import pi

# Oberes Rechteck
length1 = 2                 # Länge
width1 = 1                  # Breite
height1 = 0.3             # Höhe

# Unteres Rechteck
length2 = 2                 # Länge
width2 = 1                  # Breite
height2 = 0.5               # Höhe
offset_x = 2             # Verschiebung in X_Richtung

# Lochparameter
hole_radius = 0.2         # Radius
hole_offset_x = 0.75        # Verschiebung in X-Richtung
hole_offset_y = -0.0        # Verschiebung in Y-Richtung

# Schraube
r1 = 0.1                    # Radius der Schacht
h1 = 1.0                    # Höhe der Schacht
r2 = 0.15                   # Radius der Hülse
h2 = 0.5                    # Höhe der Hülse
versatz2 = -h2
r3 = 0.2                    # Radius des Kopfes
h3 = 0.2                    # Höhe des Kopfes
geometry_offset_z = -0.5    # Verschiebung der CShraube in Z-Richtung


def create_box(length, width, height, center=[0, 0, 0]):
    lx, ly, lz = length / 2, width / 2, height / 2
    cx, cy, cz = center
    vertices = np.array([
        [-lx, -ly, -lz],
        [ lx, -ly, -lz],
        [ lx,  ly, -lz],
        [-lx,  ly, -lz],
        [-lx, -ly,  lz],
        [ lx, -ly,  lz],
        [ lx,  ly,  lz],
        [-lx,  ly,  lz],
    ], dtype=np.float32) + np.array(center, dtype=np.float32)

    faces = np.array([
        [0, 1, 2], [0, 2, 3],
        [4, 5, 6], [4, 6, 7],
        [0, 1, 5], [0, 5, 4],
        [1, 2, 6], [1, 6, 5],
        [2, 3, 7], [2, 7, 6],
        [3, 0, 4], [3, 4, 7]
    ], dtype=np.uint32)

    return vertices, faces

def create_cylinder(radius, height, segments=100, center=[0, 0, 0]):
    angle = np.linspace(0, 2 * pi, segments)
    x = radius * np.cos(angle)
    y = radius * np.sin(angle)
    z_bottom = np.zeros_like(x) - height / 2
    z_top = np.zeros_like(x) + height / 2

    bottom_circle = np.stack((x, y, z_bottom), axis=1)
    top_circle = np.stack((x, y, z_top), axis=1)

    vertices = np.vstack((bottom_circle, top_circle)) + np.array(center, dtype=np.float32)
    faces = []

    for i in range(segments):
        next_i = (i + 1) % segments
        faces.append([i, next_i, segments + next_i])
        faces.append([i, segments + next_i, segments + i])

    return vertices, np.array(faces, dtype=np.uint32)

def zylinder(r, h, center_z=0, deckel=True, n=100):
    phi = np.linspace(0, 2 * np.pi, n, endpoint=False)
    x, y = r * np.cos(phi), r * np.sin(phi)
    z0 = np.full(n, center_z)
    z1 = z0 + h

    unten = np.stack([x, y, z0], axis=1)
    oben = np.stack([x, y, z1], axis=1)
    verts = np.vstack([unten, oben])
    faces = []

    for i in range(n):
        j = (i + 1) % n
        faces += [[i, j, j + n], [i, j + n, i + n]]
        if deckel:
            faces += [[2 * n, j, i], [2 * n + 1, i + n, j + n]]

    if deckel:
        center = np.array([[0, 0, center_z], [0, 0, center_z + h]])
        verts = np.vstack([verts, center])

    return verts.astype(np.float32), np.array(faces, dtype=np.uint32)

def huelse(r_in, r_out, h, z_off=0, n=100):
    phi = np.linspace(0, 2 * np.pi, n, endpoint=False)
    cos, sin = np.cos(phi), np.sin(phi)
    zi = np.full(n, z_off)
    zo = zi + h

    def ring(r): return np.stack([r * cos, r * sin], axis=1)

    vin0 = np.c_[ring(r_in), zi]
    vin1 = np.c_[ring(r_in), zo]
    vout0 = np.c_[ring(r_out), zi]
    vout1 = np.c_[ring(r_out), zo]

    verts = np.vstack([vin0, vin1, vout0, vout1])
    faces = []

    for i in range(n):
        j = (i + 1) % n
        faces += [[i, j, n + j], [i, n + j, n + i]]
        o = 2 * n
        faces += [[o + j, o + i, o + n + i], [o + j, o + n + i, o + n + j]]
        faces += [[n + i, n + j, o + n + j], [n + i, o + n + j, o + n + i]]

    return verts.astype(np.float32), np.array(faces, dtype=np.uint32)


def visualize_complete_scene(
    length1, width1, height1,
    length2, width2, height2,
    hole_radius,
    offset_x,
    hole_offset_x, hole_offset_y,
    geometry_offset_z
):
    plot = k3d.plot()

    center1 = [0, 0, height1 / 2]
    box1_vertices, box1_faces = create_box(length1, width1, height1, center=center1)
    plot += k3d.mesh(vertices=box1_vertices, indices=box1_faces, color=0x00ff00, opacity=1)

    center2 = [offset_x, 0, -height2 / 2]
    box2_vertices, box2_faces = create_box(length2, width2, height2, center=center2)
    plot += k3d.mesh(vertices=box2_vertices, indices=box2_faces, color=0x00ff00, opacity=1)

    total_height = height1 + height2 + 0.1
    hole_center_z = (height1 - height2) / 2
    hole_center = [hole_offset_x, hole_offset_y, hole_center_z]
    cyl_vertices, cyl_faces = create_cylinder(hole_radius, total_height, center=hole_center)
    plot += k3d.mesh(vertices=cyl_vertices, indices=cyl_faces, color=0xff0000, opacity=1)

    geom_center = [hole_center[0], hole_center[1], hole_center[2] + geometry_offset_z]

    v1, f1 = zylinder(r1, h1)
    v1 += np.array(geom_center)

    v2, f2 = huelse(r1, r2, h2, z_off=h1 + versatz2)
    v2 += np.array(geom_center)

    v3, f3 = zylinder(r3, h3, center_z=h1)
    v3 += np.array(geom_center)

    plot += k3d.mesh(v1, f1, color=0x696969)
    plot += k3d.mesh(v2, f2, color=0x000000, opacity=1)
    plot += k3d.mesh(v3, f3, color=0x696969)

    plot.display()

visualize_complete_scene(
    length1, width1, height1,
    length2, width2, height2,
    hole_radius,
    offset_x,
    hole_offset_x, hole_offset_y,
    geometry_offset_z
)

/Users/gui/Documents/VSCode/bmcs_env/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [1]:
import numpy as np
import k3d
import cadquery as cq
from cadquery import exporters

def compute_symmetric_positions(start, end, spacing):
    length = end - start
    n_bars = int(np.floor(length / spacing)) + 1
    offset = (length - (n_bars - 1) * spacing) / 2
    return [start + offset + i * spacing for i in range(n_bars)]

def create_rebar_lines(l, b, t, spacing_x, spacing_y, layer_positions, A_bar_x, A_bar_y, cover=0):
    lines_x, lines_y = [], []
    x_start, x_end = -l / 2 + cover / 1000, l / 2 - cover / 1000
    y_start, y_end = -b / 2 + cover / 1000, b / 2 - cover / 1000

    x_positions = compute_symmetric_positions(x_start, x_end, spacing_x / 1000)
    y_positions = compute_symmetric_positions(y_start, y_end, spacing_y / 1000)

    total_rebars = {}

    for z_mm in layer_positions:
        z = z_mm / 1000
        count_x, count_y = 0, 0
        for y in y_positions:
            lines_x.append(np.array([[x_start, y, z], [x_end, y, z]], dtype=np.float32))
            count_x += 1
        for x in x_positions:
            lines_y.append(np.array([[x, y_start, z], [x, y_end, z]], dtype=np.float32))
            count_y += 1
        total_rebars[z_mm] = {
            "x": count_x,
            "y": count_y,
            "A_x": count_x * A_bar_x,
            "A_y": count_y * A_bar_y,
            "total": count_x + count_y,
            "A_total": count_x * A_bar_x + count_y * A_bar_y
        }

    return lines_x, lines_y, total_rebars

def lines_to_cq_edges(lines):
    return [cq.Edge.makeLine(cq.Vector(*line[0]), cq.Vector(*line[1])) for line in lines]

def plot_and_export_with_rebars(l, b, t, a, spacing_x, spacing_y,
                                 layer_positions, A_bar_x, A_bar_y,
                                 cover, export,
                                 geometry_opacity):
    l_m, b_m, t_m, a_m = l / 1000, b / 1000, t / 1000, a / 1000

    # Define segment lengths and positions
    end_length = a_m
    center_length = (l_m - 2 * a_m) / 2
    lengths = [end_length, center_length, center_length, end_length]
    x_positions = np.cumsum([0] + lengths) - l_m / 2

    # Start from single base block
    base_block = cq.Workplane("XY").box(l_m, b_m, t_m).translate((0, 0, t_m / 2))

    # Cut the block into 4 partitioned solids
    partitions = []
    for i in range(4):
        seg_length = lengths[i]
        x_center = (x_positions[i] + x_positions[i+1]) / 2
        cutter = cq.Workplane("XY").box(seg_length, b_m + 0.01, t_m + 0.01).translate((x_center, 0, t_m / 2))
        part = base_block.intersect(cutter)
        partitions.append(part.val())

    # Visualization
    plot = k3d.plot()
    all_vertices = []
    colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00]
    for solid, color in zip(partitions, colors):
        mesh = solid.tessellate(0.001)
        vertices = np.array([[v.x, v.y, v.z] for v in mesh[0]], dtype=np.float32)
        indices = np.array(mesh[1], dtype=np.uint32)
        all_vertices.append(vertices)
        plot += k3d.mesh(vertices=vertices, indices=indices, color=color, opacity=geometry_opacity)

    all_vertices_np = np.vstack(all_vertices)
    center = (all_vertices_np.max(axis=0) + all_vertices_np.min(axis=0)) / 2
    diag = np.linalg.norm(all_vertices_np.max(axis=0) - all_vertices_np.min(axis=0))
    plot.camera_auto_fit = False
    plot.camera_position = (center + np.array([1.5, 1.5, 1.2]) * diag).tolist()
    plot.camera_look_at = center.tolist()

    # Rebars
    rebars_x, rebars_y, total_rebars = create_rebar_lines(
        l_m, b_m, t_m, spacing_x, spacing_y,
        layer_positions, A_bar_x, A_bar_y, cover
    )

    for line in rebars_x:
        plot += k3d.line(line, shader="simple", width=0.005, color=0x111111)
    for line in rebars_y:
        plot += k3d.line(line, shader="simple", width=0.005, color=0x222222)

    print("\nRebar count and areas per layer (X/Y directions):")
    A_total_x = A_total_y = 0.0
    for z, counts in total_rebars.items():
        print(f"Layer at z={z:.1f} mm -> X: {counts['x']} (A={counts['A_x']:.2f} mm²), "
              f"Y: {counts['y']} (A={counts['A_y']:.2f} mm²), "
              f"Total A={counts['A_total']:.2f} mm²")
        A_total_x += counts['A_x']
        A_total_y += counts['A_y']

    A_geom = b * t
    A_reinf = A_total_x + A_total_y
    rho = A_reinf / A_geom

    print("\nTotal reinforcement summary:")
    print(f"  Total A_x: {A_total_x:.2f} mm²")
    print(f"  Total A_y: {A_total_y:.2f} mm²")
    print(f"  Geometry A_geom (Y×Z): {A_geom:.2f} mm²")
    print(f"  Reinforcement ratio ρ: {rho:.6f} (-)")

    plot.display()

    if export:
        # Export full compound of boolean-cut solids
        compound = cq.Compound.makeCompound(partitions)
        exporters.export(compound, 'partitioned_closed_parts_fixed.step', exportType='STEP')
        print("✅ Geometry exported: partitioned_closed_parts_fixed.step")

        edges_x = lines_to_cq_edges(rebars_x)
        edges_y = lines_to_cq_edges(rebars_y)
        exporters.export(cq.Compound.makeCompound(edges_x), 'rebars_x.step', exportType='STEP')
        exporters.export(cq.Compound.makeCompound(edges_y), 'rebars_y.step', exportType='STEP')
        print("✅ Rebars exported: rebars_x.step, rebars_y.step")
    else:
        print("⚠️ STEP export skipped (export=False)")

# Example usage
plot_and_export_with_rebars(
    l=2200,               # mm
    b=120,                # mm
    t=15,                 # mm
    a=100,                # mm
    spacing_x=16,         # mm
    spacing_y=12.7,       # mm
    layer_positions=[5, 7.5, 12],  # mm from bottom
    A_bar_x=1.9,          # mm² per rebar in X
    A_bar_y=0.95,         # mm² per rebar in Y
    cover=5,              # mm
    export=True,
    geometry_opacity=0.3
)


Rebar count and areas per layer (X/Y directions):
Layer at z=5.0 mm -> X: 9 (A=17.10 mm²), Y: 137 (A=130.15 mm²), Total A=147.25 mm²
Layer at z=7.5 mm -> X: 9 (A=17.10 mm²), Y: 137 (A=130.15 mm²), Total A=147.25 mm²
Layer at z=12.0 mm -> X: 9 (A=17.10 mm²), Y: 137 (A=130.15 mm²), Total A=147.25 mm²

Total reinforcement summary:
  Total A_x: 51.30 mm²
  Total A_y: 390.45 mm²
  Geometry A_geom (Y×Z): 1800.00 mm²
  Reinforcement ratio ρ: 0.245417 (-)


Output()

✅ Geometry exported: partitioned_closed_parts_fixed.step
✅ Rebars exported: rebars_x.step, rebars_y.step


In [2]:
E = 200000 # MPa
t = 15 # mm
disp = 200 # mm
L = 2000 # mm
b = 120 # mm
# Berechnung der Last

load = 4 * E * b* t**3 * disp / (L**3)  # Last in N
load 


8100.0

In [3]:
import numpy as np

# Young’s-modulus values in MPa
E_modul = np.array([100, 120, 140, 160, 180, 200, 220, 240, 260],
                   dtype=float)

fnmk = 1200  # σ (or f_nk) in MPa

# ε in mm / m  (= 1000 × dimensionless strain)
eps = fnmk / (E_modul * 1000.0)

print(eps)

[0.012      0.01       0.00857143 0.0075     0.00666667 0.006
 0.00545455 0.005      0.00461538]
